In [1]:
import pandas as pd
import numpy as np
import os
import json

# 读取数据
root_path = "/workspace/med_proj"
file_path = f"{root_path}/data/all_data.xlsx"
data = pd.read_excel(file_path)

id2cluster = pd.read_csv(f"{root_path}/data/result.csv", index_col=None, header=0)
with open(f"{root_path}/results/dataprocess/name_id_mapping.json", "r", encoding="utf-8") as f:
    name2id = json.load(f)
id2name = {v: k for k, v in name2id.items()}

name2cluster = {}
for index, row in id2cluster.iterrows():
    name2cluster[id2name[row['ID']]] = row['cluster']

In [2]:
df_filtered = data[data['姓名'].isin(name2id.keys())]

# 将 name2id 中的对应ID添加到新列中
df_filtered['cluster'] = df_filtered['姓名'].map(name2cluster)
df_filtered.rename(columns={'姓名': 'name'}, inplace=True)
df_filtered['性别'] = df_filtered['性别'].replace({'男': 0, '女': 1})
df_filtered['转归'] = df_filtered['转归'].replace({'存活': 1,'死亡': 0, '好转': 1, "未愈": 0, "其他":1})
df_filtered['机械通气'] = df_filtered['机械通气'].replace({'无': 0, '有': 1})
df_filtered['是否二感'] = df_filtered['是否二感'].replace({'否': 0, '是': 1, '疑似': 0})

/tmp/ipykernel_3829704/3662610358.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['cluster'] = df_filtered['姓名'].map(name2cluster)
/tmp/ipykernel_3829704/3662610358.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.rename(columns={'姓名': 'name'}, inplace=True)
/tmp/ipykernel_3829704/3662610358.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('f

In [3]:
df_filtered.to_csv(f'{root_path}/results/anova/cluster_feature_adjust.csv', index=False)

### 使用方差分析得到结结果

In [5]:
from scipy.stats import f_oneway
import numpy as np

data = df_filtered
# Extracting columns for ANOVA, excluding 'name' and 'cluster' columns
anova_columns = data.columns.drop(['name', 'cluster'])

# Dictionary to hold ANOVA results
anova_results = {}

# Performing ANOVA for each column and storing the results
for column in anova_columns:
    groups = data.groupby('cluster')[column].apply(list)
    f_val, p_val = f_oneway(*groups)
    anova_results[column] = {'F-value': f_val, 'p-value': p_val}

# Convert ANOVA results to a DataFrame
anova_df = pd.DataFrame(anova_results).T

# Calculating mean values for each column per cluster
mean_values = data.groupby('cluster')[anova_columns].mean()


anova_df.to_csv(f'{root_path}/results/anova/anova.csv', encoding="utf-8-sig")
mean_values.to_csv(f'{root_path}/results/anova/mean_values.csv', encoding="utf-8-sig")

In [6]:
from scipy.stats import f_oneway
import pandas as pd

# 假设 df_filtered 是你的数据框
data = df_filtered

# 初始化结果存储字典
anova_results_by_cluster = {}

# 需要比较的 cluster 列表
clusters_to_compare = [1, 2, 4]

# 遍历除了'name'和'cluster'外的所有列
for column in data.columns.drop(['name', 'cluster']):
    # 存储当前列的比较结果
    column_results = {}
    
    # 对每个需要比较的 cluster 执行 ANOVA
    for compare_cluster in clusters_to_compare:
        # 提取 cluster = 3 和当前比较 cluster 的数据
        group1 = data[data['cluster'] == 3][column]
        group2 = data[data['cluster'] == compare_cluster][column]

        # 执行 ANOVA
        f_val, p_val = f_oneway(group1, group2)
        column_results[f'cluster_3_vs_{compare_cluster}'] = {'F-value': f_val, 'p-value': p_val}
    
    # 将当前列的结果存入总字典
    anova_results_by_cluster[column] = column_results

# 将结果转换为更方便阅读的形式
anova_df = pd.DataFrame.from_dict({(i, j): anova_results_by_cluster[i][j] 
                                   for i in anova_results_by_cluster.keys() 
                                   for j in anova_results_by_cluster[i].keys()},
                                  orient='index')

# 可以将结果保存为 CSV 文件
anova_df.to_csv(f'{root_path}/results/anova/cluster_3_comparisons.csv', encoding="utf-8-sig")
